# Discovering Disease Outbreaks from News Headlines

Identifying and mapping epidemics is crucial to prevent or respond to deadly disease outbreaks. Your first assignment for the WHO is as follows:

- Extract the locations (city and/or country name) from each news headline.
- Find the geographic coordinates of each headline using the city/country.
- Cluster (group) the headlines based on the geographic location.
- Visualize the clusters on a map and analyze them for patterns indicating an epidemic.
- Investigate the largest clusters for signs of disease outbreaks.
- Review headlines in the largest clusters within the United States and around the world. If any disease outbreak is   particularly dominant, visualize all worldwide mentions of that disease.
- Provide a summary of your findings to your superiors at the WHO so they can direct resources.

# 1. Parsing the News Headlines

**Objective**

Find any city and/or country names mentioned in each of the news headlines.

**Workflow**

1. Load in the headline data and examine it for any data quality issues.
1. Use any library/data structure to read in the headlines
1. Read through some of the headlines and identify potential problems
1. Using regular expressions and the cities and countries within the geonamescache library, match any cities/countries within each headline.
1. Make sure to normalize headlines and city/country names by removing accent marks. This can be done with the unidecode library.
1. Watch out for multiple cities in a headline and matches on short words! We want the match to be on the entire city—for example San Marino—and not a partial match—San.
1. Put the extracted data into a pandas DataFrame with three columns: headline, city, country.
1. Make sure there were no issues with the extraction by sampling some of the headlines and examining the city and country names.
1. One method for finding problems is to look for the most common names and see if there are any issues.
1. Once you are confident you’ve found all the cities/countries in each headline, save the DataFrame for the next part.

**Importance to project**

* We can’t do much with just the headlines; although they contain the city/country names, they do not contain the geographic information—latitude and longitude—we need to find clusters of disease outbreaks. The first step in getting the geographic information is to isolate the names.

* Later, we will use the names to find the location of each headline, which requires bringing in external data (through geonamescache).

* This workflow is common in data science. First, we separate the useful information from the noise—data mining—and then we augment it with external data—data engineering.

## Import all relevent libraries

In [1]:
# Regular expression
import re
from typing import Optional

# Data analysis and wrangling
import numpy as np
import pandas as pd

# Set display options for pandas
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("max_colwidth", None)

# Normalized unicode data (to remove accents)
import unidecode

## Visualization
# matplotlib
import matplotlib.pyplot as plt

# Ignore warning
import warnings

warnings.filterwarnings("ignore")

## GeonamesCache data


In [2]:
from geonamescache import GeonamesCache

"""
get_continents()
get_countries()
get_us_states()
get_cities()
get_countries_by_names()
get_us_states_by_names()
get_cities_by_name(name)
get_us_counties()
"""
gc = GeonamesCache()

### US states and counties 

#### US states

In [3]:
states = pd.DataFrame(gc.get_us_states_by_names()).T.reset_index(drop=True)
states = states.sort_values(by="name", ascending=False).reset_index(drop=True)
states = states.rename(
    columns={"name": "statename", "code": "statecode", "geonameid": "stateid"}
)
# states.info()
print(states.shape)
states.head()

(51, 4)


,statecode,statename,fips,stateid
0,WY,Wyoming,56,5843591
1,WI,Wisconsin,55,5279468
2,WV,West Virginia,54,4826850
3,WA,Washington,53,5815135
4,VA,Virginia,51,6254928


> 📝 **Note**: `states` dataFrame consits of 50 states and 1 district of Columbia

#### US Counties

In [4]:
# Retrive countynames data and create a dataframe.
counties = pd.DataFrame(gc.get_us_counties()).T.reset_index(drop=True).T

counties.columns = ["countyid", "countyname", "statecode"]
counties = counties.sort_values(
    by=["statecode", "countyname"], ascending=False
).reset_index(drop=True)
print(counties.shape)
counties.head(10)

(3235, 3)


,countyid,countyname,statecode
0,56045,Weston County,WY
1,56043,Washakie County,WY
2,56041,Uinta County,WY
3,56039,Teton County,WY
4,56037,Sweetwater County,WY
5,56035,Sublette County,WY
6,56033,Sheridan County,WY
7,56031,Platte County,WY
8,56029,Park County,WY
9,56027,Niobrara County,WY


In [5]:
3225 - 3143

82

> 📝 **Note**: `counties` dataFrame consists of 3,225 counties in the US and its territories. The dataframe has 6 more statecodes and 92 more counies of the US territories. 

In [6]:
counties.statecode.value_counts()

statecode
TX    254
GA    159
VA    134
KY    120
MO    115
KS    105
IL    102
NC    100
IA     99
TN     95
NE     93
IN     92
OH     88
MN     87
MI     83
MS     82
PR     78
OK     77
AR     75
WI     72
AL     67
FL     67
PA     67
SD     66
LA     64
CO     64
NY     62
CA     58
MT     56
WV     55
ND     53
SC     46
ID     44
WA     39
OR     36
NM     33
AK     29
UT     29
MD     24
WY     23
NJ     21
NV     17
ME     16
AZ     15
VT     14
MA     14
NH     10
CT      8
RI      5
HI      5
AS      5
MP      4
VI      3
DE      3
GU      1
DC      1
UM      1
Name: count, dtype: int64

In [7]:
states.statecode.nunique()

51

In [8]:
counties.statecode.nunique()

57

#### Merge US states and counties

In [9]:
# merge state and county dataframes

us_counties = (
    pd.merge(
        counties, states, on="statecode", how="left", suffixes=("_county", "_state")
    )
    .sort_values(by=["statename", "countyname"], ascending=False)
    .reset_index(drop=True)
)
us_counties = us_counties.rename(columns={"countyname": "county", "statename": "state"})

# Remove accents from county names
us_counties["county"] = us_counties["county"].apply(lambda x: unidecode.unidecode(x))

print(us_counties.shape)
us_counties.head()

(3235, 6)


,countyid,county,statecode,state,fips,stateid
0,56045,Weston County,WY,Wyoming,56,5843591
1,56043,Washakie County,WY,Wyoming,56,5843591
2,56041,Uinta County,WY,Wyoming,56,5843591
3,56039,Teton County,WY,Wyoming,56,5843591
4,56037,Sweetwater County,WY,Wyoming,56,5843591


In [10]:
# How many counties and states that have no state code in `states` dataframe?
us_counties[us_counties.state.isna()]

,countyid,county,statecode,state,fips,stateid
3143,72153,Yauco Municipio,PR,NaN,NaN,NaN
3144,72151,Yabucoa Municipio,PR,NaN,NaN,NaN
3145,60050,Western District,AS,NaN,NaN,NaN
3146,72149,Villalba Municipio,PR,NaN,NaN,NaN
3147,72147,Vieques Municipio,PR,NaN,NaN,NaN
3148,72145,Vega Baja Municipio,PR,NaN,NaN,NaN
3149,72143,Vega Alta Municipio,PR,NaN,NaN,NaN
3150,72141,Utuado Municipio,PR,NaN,NaN,NaN
3151,72139,Trujillo Alto Municipio,PR,NaN,NaN,NaN
3152,72137,Toa Baja Municipio,PR,NaN,NaN,NaN


In [11]:
def remove_county_suffix(county: str) -> Optional[str]:
    """Remove common suffixes from county names."""
    s = [
        "County",
        "Municipality",
        "Municipio",
        "Census Area",
        "City and Borough",
        "Borough",
        "Parish",
    ]
    #  Create a regex pattern to match any of the suffixes

    regexs = "|".join(s)

    # Use regex to remove the suffixes
    county_name_only = re.sub(rf"\s*({regexs})\s*$", "", county, flags=re.IGNORECASE)
    return county_name_only.strip() if county_name_only else None


# Insert a new column for cleaned county names
us_counties.insert(loc=2, column="countyname", value=None)

# Apply the function to the 'county' column
us_counties["countyname"] = us_counties["county"].apply(remove_county_suffix)


print(counties.shape)


us_counties.head(10)

(3235, 3)


,countyid,county,countyname,statecode,state,fips,stateid
0,56045,Weston County,Weston,WY,Wyoming,56,5843591
1,56043,Washakie County,Washakie,WY,Wyoming,56,5843591
2,56041,Uinta County,Uinta,WY,Wyoming,56,5843591
3,56039,Teton County,Teton,WY,Wyoming,56,5843591
4,56037,Sweetwater County,Sweetwater,WY,Wyoming,56,5843591
5,56035,Sublette County,Sublette,WY,Wyoming,56,5843591
6,56033,Sheridan County,Sheridan,WY,Wyoming,56,5843591
7,56031,Platte County,Platte,WY,Wyoming,56,5843591
8,56029,Park County,Park,WY,Wyoming,56,5843591
9,56027,Niobrara County,Niobrara,WY,Wyoming,56,5843591


### Countries and cities


#### Countries

In [12]:
countries = pd.DataFrame(gc.get_countries_by_names()).T
# sort countries by name and reset index
countries = countries.sort_values(by="name", ascending=True).reset_index(drop=True)
# Rename columns for clarity
countries.rename(
    columns={"iso": "countrycode", "name": "countryname", "geonameid": "countryid"},
    inplace=True,
)

# countries.info()
print(countries.shape)
countries.head(10)

(252, 17)


,countryid,countryname,countrycode,iso3,isonumeric,fips,continentcode,capital,areakm2,population,tld,currencycode,currencyname,phone,postalcoderegex,languages,neighbours
0,1149361,Afghanistan,AF,AFG,4,AF,AS,Kabul,647500,37172386,.af,AFN,Afghani,93,,"fa-AF,ps,uz-AF,tk","TM,CN,IR,TJ,PK,UZ"
1,661882,Aland Islands,AX,ALA,248,,EU,Mariehamn,1580,26711,.ax,EUR,Euro,+358-18,^(?:FI)*(\d{5})$,sv-AX,
2,783754,Albania,AL,ALB,8,AL,EU,Tirana,28748,2866376,.al,ALL,Lek,355,^(\d{4})$,"sq,el","MK,GR,ME,RS,XK"
3,2589581,Algeria,DZ,DZA,12,AG,AF,Algiers,2381740,42228429,.dz,DZD,Dinar,213,^(\d{5})$,ar-DZ,"NE,EH,LY,MR,TN,MA,ML"
4,5880801,American Samoa,AS,ASM,16,AQ,OC,Pago Pago,199,55465,.as,USD,Dollar,+1-684,96799,"en-AS,sm,to",
5,3041565,Andorra,AD,AND,20,AN,EU,Andorra la Vella,468,77006,.ad,EUR,Euro,376,^(?:AD)*(\d{3})$,ca,"ES,FR"
6,3351879,Angola,AO,AGO,24,AO,AF,Luanda,1246700,30809762,.ao,AOA,Kwanza,244,,pt-AO,"CD,NA,ZM,CG"
7,3573511,Anguilla,AI,AIA,660,AV,NA,The Valley,102,13254,.ai,XCD,Dollar,+1-264,,en-AI,
8,6697173,Antarctica,AQ,ATA,10,AY,AN,,14000000,0,.aq,,,,,,
9,3576396,Antigua and Barbuda,AG,ATG,28,AC,NA,St. John's,443,96286,.ag,XCD,Dollar,+1-268,,en-AG,


In [13]:
# Keep only nessessary columns
countries = countries[["countryname", "capital", "countrycode", "countryid"]]

countries.head(10)

,countryname,capital,countrycode,countryid
0,Afghanistan,Kabul,AF,1149361
1,Aland Islands,Mariehamn,AX,661882
2,Albania,Tirana,AL,783754
3,Algeria,Algiers,DZ,2589581
4,American Samoa,Pago Pago,AS,5880801
5,Andorra,Andorra la Vella,AD,3041565
6,Angola,Luanda,AO,3351879
7,Anguilla,The Valley,AI,3573511
8,Antarctica,,AQ,6697173
9,Antigua and Barbuda,St. John's,AG,3576396


#### Cities


In [14]:
# create a dataframe for cities
cities = pd.DataFrame(gc.get_cities()).T.reset_index(drop=True)
print(cities.shape)
cities.head()

(26463, 9)


,geonameid,name,latitude,longitude,countrycode,population,timezone,admin1code,alternatenames
0,3040051,les Escaldes,42.50729,1.53414,AD,15853,Europe/Andorra,08,"[Ehskal'des-Ehndzhordani, Escaldes, Escaldes-Engordany, Les Escaldes, esukarudesu=engorudani jiao qu, lai sai si ka er de-en ge er da, Эскальдес-Энджордани, エスカルデス＝エンゴルダニ教区, 萊塞斯卡爾德-恩戈爾達, 萊塞斯卡爾德－恩戈爾達]"
1,3041563,Andorra la Vella,42.50779,1.52109,AD,20430,Europe/Andorra,07,"[ALV, Ando-la-Vyey, Andora, Andora la Vela, Andora la Velja, Andora lja Vehl'ja, Andoro Malnova, Andorra, Andorra Tuan, Andorra a Vella, Andorra la Biella, Andorra la Vella, Andorra la Vielha, Andorra-a-Velha, Andorra-la-Vel'ja, Andorra-la-Vielye, Andorre-la-Vieille, Andò-la-Vyèy, Andòrra la Vièlha, an dao er cheng, andolalabeya, andwra la fyla, Ανδόρρα, Андора ла Веля, Андора ла Веља, Андора ля Вэлья, Андорра-ла-Велья, אנדורה לה וולה, أندورا لا فيلا, አንዶራ ላ ቬላ, アンドラ・ラ・ヴェリャ, 安道爾城, 안도라라베야]"
2,290594,Umm Al Quwain City,25.56473,55.55517,AE,62747,Asia/Dubai,07,"[Oumm al Qaiwain, Oumm al Qaïwaïn, Um al Kawain, Um al Quweim, Umm Al Quwain City, Umm al Qaiwain, Umm al Qawain, Umm al Qaywayn, Umm al-Quwain, Umm-ehl'-Kajvajn, Yumul al Quwain, am alqywyn, mdynt am alqywyn, Умм-эль-Кайвайн, أم القيوين, مدينة ام القيوين]"
3,291074,Ras Al Khaimah City,25.78953,55.9432,AE,351943,Asia/Dubai,05,"[Julfa, Khaimah, RAK City, RKT, Ra's al Khaymah, Ra's al-Chaima, Ras Al Khaimah City, Ras al Khaimah, Ras al-Khaimah, Ras el Khaimah, Ras el Khaïmah, Ras el-Kheima, Ras-ehl'-Khajma, Ra’s al Khaymah, Ra’s al-Chaima, mdynt ras alkhymt, ras alkhymt, Рас-эль-Хайма, رأس الخيمة, مدينة رأس الخيمة]"
4,291580,Zayed City,23.65416,53.70522,AE,63482,Asia/Dubai,01,"[Bid' Zayed, Bid’ Zayed, Madinat Za'id, Madinat Zayid, Madīnat Zāyid, Madīnat Zā’id, Zayed City, mdynt zayd, مدينة زايد]"


In [15]:
# sort cities by country code and name in descending order
cities = cities.sort_values(by=["countrycode", "name"]).reset_index(drop=True)
# drop unnecessary columns
cities = cities.drop(columns=["population", "alternatenames"])
# rename columns for clarity
cities = cities.rename({"geonameid": "cityid", "name": "cityname_raw"}, axis=1)
# transform all accented strings to English alphabets'
cities.insert(loc=2, column="cityname", value=np.nan)
cities.cityname = cities["cityname_raw"].apply(
    func=lambda x: unidecode.unidecode(string=x)
)

cities.head(10)

,cityid,cityname_raw,cityname,latitude,longitude,countrycode,timezone,admin1code
0,3041563,Andorra la Vella,Andorra la Vella,42.50779,1.52109,AD,Europe/Andorra,07
1,3040051,les Escaldes,les Escaldes,42.50729,1.53414,AD,Europe/Andorra,08
2,292968,Abu Dhabi,Abu Dhabi,24.45118,54.39696,AE,Asia/Dubai,01
3,292953,Adh Dhayd,Adh Dhayd,25.28812,55.88157,AE,Asia/Dubai,06
4,292932,Ajman City,Ajman City,25.40177,55.47878,AE,Asia/Dubai,02
5,292913,Al Ain City,Al Ain City,24.19167,55.76056,AE,Asia/Dubai,01
6,292878,Al Fujairah City,Al Fujairah City,25.11641,56.34141,AE,Asia/Dubai,04
7,12047416,Al Shamkhah City,Al Shamkhah City,24.39268,54.70779,AE,Asia/Dubai,01
8,292688,Ar Ruways,Ar Ruways,24.11028,52.73056,AE,Asia/Dubai,01
9,12042052,Bani Yas City,Bani Yas City,24.30978,54.62944,AE,Asia/Dubai,01


#### Merge `countries` dataframes to `cities` dataframe 

In [16]:
# How many different country codes are there in the cities dataframe?
len(cities.countrycode.unique())

244

> ⚠️ **Caution**: The `countries` and `cities` dataframes do not have the same number of countries. The `countries` dataframe has 252 countries, but `cities` dataframe has only 244 countries. Because of some countries not having cities in the `cities` df, we will use an outer join to merge the two dataframes. This will ensure that all countries are included, even if they do not have any cities listed in the `cities` df.


In [17]:
city = (
    cities[["cityname", "latitude", "longitude", "countrycode", "cityid"]]
    .merge(countries, how="left", on="countrycode", suffixes=("_city", "_country"))
    .sort_values(by=["countrycode", "cityname"], ascending=True)
    .reset_index(drop=True)
)
# rearrange columns for better readability
city = city[
    [
        "cityname",
        "countryname",
        "capital",
        "latitude",
        "longitude",
        "countrycode",
        "cityid",
        "countryid",
    ]
]

print(city.shape)
city.head(10)

(26463, 8)


,cityname,countryname,capital,latitude,longitude,countrycode,cityid,countryid
0,Andorra la Vella,Andorra,Andorra la Vella,42.50779,1.52109,AD,3041563,3041565
1,les Escaldes,Andorra,Andorra la Vella,42.50729,1.53414,AD,3040051,3041565
2,Abu Dhabi,United Arab Emirates,Abu Dhabi,24.45118,54.39696,AE,292968,290557
3,Adh Dhayd,United Arab Emirates,Abu Dhabi,25.28812,55.88157,AE,292953,290557
4,Ajman City,United Arab Emirates,Abu Dhabi,25.40177,55.47878,AE,292932,290557
5,Al Ain City,United Arab Emirates,Abu Dhabi,24.19167,55.76056,AE,292913,290557
6,Al Fujairah City,United Arab Emirates,Abu Dhabi,25.11641,56.34141,AE,292878,290557
7,Al Shamkhah City,United Arab Emirates,Abu Dhabi,24.39268,54.70779,AE,12047416,290557
8,Ar Ruways,United Arab Emirates,Abu Dhabi,24.11028,52.73056,AE,292688,290557
9,Bani Yas City,United Arab Emirates,Abu Dhabi,24.30978,54.62944,AE,12042052,290557


In [18]:
# city.columns.tolist()

## Headline data


In [19]:
file_contents = ""
with open("headlines.txt", "r") as file_handle:
    for line in file_handle.readlines():
        file_contents += line.replace("-", " ")
        unidecode.unidecode(line)

# View first 500 charactors
file_contents[:500]

'Zika Outbreak Hits Miami\nCould Zika Reach New York City?\nFirst Case of Zika in Miami Beach\nMystery Virus Spreads in Recife, Brazil\nDallas man comes down with case of Zika\nTrinidad confirms first Zika case\nZika Concerns are Spreading in Houston\nGeneve Scientists Battle to Find Cure\nThe CDC in Atlanta is Growing Worried\nZika Infested Monkeys in Sao Paulo\nBrownsville teen contracts Zika virus\nMosquito control efforts in St. Louis take new tactics with Zika threat\nSan Juan reports 1st U.S. Zika rela'

In [26]:
# Create dataframe of headlines
df = (
    pd.DataFrame(file_contents.split("\n"), columns=["headline"])
    .sort_values(by="headline", ascending=True)
    .reset_index(drop=True)
)
print(df.shape)
df.head(10)

(650, 1)


,headline
0,18 new Zika Cases in Bogota
1,19 new Zika Cases in Sengkang
2,Alameda Residents Recieve Rabies vaccine
3,Albany Residents Recieve Respiratory Syncytial Virus vaccine
4,Antipolo under threat from Zika Virus
5,Arhus is infested with Bronchitis
6,Arvada is infested with Syphilis
7,Authorities a Miami
8,Authorities are Worried about the Spread of Bronchitis in Silver Spring
9,Authorities are Worried about the Spread of Chickenpox in Hemet


In [27]:
df

,headline
0,18 new Zika Cases in Bogota
1,19 new Zika Cases in Sengkang
2,Alameda Residents Recieve Rabies vaccine
3,Albany Residents Recieve Respiratory Syncytial Virus vaccine
4,Antipolo under threat from Zika Virus
5,Arhus is infested with Bronchitis
6,Arvada is infested with Syphilis
7,Authorities a Miami
8,Authorities are Worried about the Spread of Bronchitis in Silver Spring
9,Authorities are Worried about the Spread of Chickenpox in Hemet


### Removes duplicates

In [28]:
# Check for duplicate headlines
df[df.headline.duplicated(False)]

,headline
421,Spanish Flu Outbreak in Lisbon
422,Spanish Flu Outbreak in Lisbon
424,Spanish Flu Spreading through Madrid
425,Spanish Flu Spreading through Madrid


In [30]:
# remove duplicates
df.drop_duplicates(subset="headline", inplace=True, ignore_index=True)
print(df.shape)

(648, 1)


### Add headline IDs

In [ ]:
#  use index number as headlineid
df.reset_index(drop=False, names="headlineid", inplace=True)

In [45]:
df.headlineid = df.headlineid.astype(str)
# Zero pad headlineid to 3 digits
df.headlineid = df.headlineid.str.zfill(3)

In [46]:
df

,headlineid,headline
0,000,18 new Zika Cases in Bogota
1,001,19 new Zika Cases in Sengkang
2,002,Alameda Residents Recieve Rabies vaccine
3,003,Albany Residents Recieve Respiratory Syncytial Virus vaccine
4,004,Antipolo under threat from Zika Virus
5,005,Arhus is infested with Bronchitis
6,006,Arvada is infested with Syphilis
7,007,Authorities a Miami
8,008,Authorities are Worried about the Spread of Bronchitis in Silver Spring
9,009,Authorities are Worried about the Spread of Chickenpox in Hemet


## Extract cities and countries from headlines

### Use regex to retrive all country and city names from headline column

In [47]:
from re import Pattern
from typing import Any, Optional, Iterable


# def get_compiled_regex(name: str) -> Pattern[Any]:
#     """Create a regex pattern for the given name. Compiled one name at a time."""
#     regex = f"\\b{name}\\b"
#     return re.compile(regex)  # re.IGNORECASE)


def get_big_compiled_regex(namelist: Iterable) -> Pattern[Any]:
    """
    Create a regex pattern for the given name list.
    Ensures input is a list, sorts by length (descending), then compiles.
    """
    if not isinstance(namelist, list):
        namelist = list(namelist)
    namelist.sort(key=len, reverse=True)
    all_names = "|".join(re.escape(name) for name in namelist)
    regex = f"\\b(?:{all_names})\\b"
    # return re.compile(regex, re.IGNORECASE)
    return re.compile(regex)

In [48]:
# def get_matched_country(headline: str) -> Optional[str]:
#     """Extract the matched country string from a headline string by compiled one regex pattern of country at a time."""
#     # Create a compiled regex for each country name
#     for country_name in city.countryname:
#         compiled_country = get_compiled_regex(country_name)
#         # match country in headline
#         match_country = compiled_country.search(headline)
#         if match_country is not None:
#             start, end = match_country.start(), match_country.end()
#             return headline[start:end]


def get_matched_country(headline: str) -> Optional[str]:
    """Extract the matched country name string from a headline string by compiled regex pattern of all countries."""
    compiled_countries = get_big_compiled_regex(city.countryname)
    matched_countries = re.findall(compiled_countries, headline)
    return matched_countries[0] if matched_countries else None


def get_matched_city(headline: str) -> Optional[str]:
    """Extract the matched city name string from a headline string by compiled regex pattern of all cities."""
    compiled_cities = get_big_compiled_regex(city.cityname)
    matched_cities = re.findall(compiled_cities, headline)
    return matched_cities[0] if matched_cities else None


def get_matched_usstate(headline: str) -> Optional[str]:
    """Extract the matched US state name string from a headline string by compiled regex pattern of all US states."""
    compiled_states = get_big_compiled_regex(states.statename)
    matched_states = re.findall(compiled_states, headline)
    return matched_states[0] if matched_states else None


def get_matched_uscounty(headline: str) -> Optional[str]:
    """Extract the matched US county name string from a headline string by compiled regex pattern of all US counties."""
    compiled_counties = get_big_compiled_regex(us_counties.countyname)
    matched_counties = re.findall(compiled_counties, headline)
    return matched_counties[0] if matched_counties else None

    # """Extract the matched city string from a regex match object."""
    # compiled_city = get_city_regex(city_name)
    # match_city = compiled_city.search(headline)
    # if match_city is not None:
    #     start, end = match_city.start(), match_city.end()
    #     return headline[start:end]

### Cities extraction


In [49]:
df["city"] = df.headline.apply(func=lambda x: get_matched_city(x))

print(df.shape)
df.head(10)

(648, 3)


,headlineid,headline,city
0,000,18 new Zika Cases in Bogota,Bogota
1,001,19 new Zika Cases in Sengkang,Sengkang
2,002,Alameda Residents Recieve Rabies vaccine,Alameda
3,003,Albany Residents Recieve Respiratory Syncytial Virus vaccine,Albany
4,004,Antipolo under threat from Zika Virus,Antipolo
5,005,Arhus is infested with Bronchitis,Arhus
6,006,Arvada is infested with Syphilis,Arvada
7,007,Authorities a Miami,Miami
8,008,Authorities are Worried about the Spread of Bronchitis in Silver Spring,Silver Spring
9,009,Authorities are Worried about the Spread of Chickenpox in Hemet,Hemet


> 📝 **Note**:
> The `cities` extraction is done using regex to match the city names in the headlines. The regex pattern and matching sequence (lengthiest name first) is designed to capture full city names, avoiding partial matches. 
>
> - of total 648 headlines, 
>   - 606 headlines (93.5%) have city names extracted.
>   - 42 headlines (6.5%) do not have any city names extracted.    





In [52]:
df

,headlineid,headline,city
0,000,18 new Zika Cases in Bogota,Bogota
1,001,19 new Zika Cases in Sengkang,Sengkang
2,002,Alameda Residents Recieve Rabies vaccine,Alameda
3,003,Albany Residents Recieve Respiratory Syncytial Virus vaccine,Albany
4,004,Antipolo under threat from Zika Virus,Antipolo
5,005,Arhus is infested with Bronchitis,Arhus
6,006,Arvada is infested with Syphilis,Arvada
7,007,Authorities a Miami,Miami
8,008,Authorities are Worried about the Spread of Bronchitis in Silver Spring,Silver Spring
9,009,Authorities are Worried about the Spread of Chickenpox in Hemet,Hemet


In [50]:
print(len(df[df.city.isnull()]))
df[df.city.isnull()]

42


,headlineid,headline,city
45,045,Bronchitis Outbreak in Manhasset,None
52,052,Carnival under threat in Rio De Janeiro due to Zika outbreak,None
72,072,Chikungunya has not Left Pismo Beach,None
99,099,Fort Belvoir tests new cure for Hepatitis C,None
111,111,Greenwich Establishes Zika Task Force,None
113,113,Gympie Patient in Critical Condition after Contracting Chlamydia,None
182,182,Longwood volunteers spreading Zika awareness,None
183,183,Louisiana Zika cases up to 26,None
207,207,Maka City Experiences Influenza Outbreak,None
208,208,Malaria Exposure in Sussex,None


### Add country names column to the DataFrame by merging with `city` dataframe 

In [56]:
merged = df.merge(
    right=city[["countryname", "cityname", "cityid"]],
    how="left",
    left_on="city",
    right_on="cityname",
    suffixes=("_headline", "_city"),
)

merged

,headlineid,headline,city,countryname,cityname,cityid
0,000,18 new Zika Cases in Bogota,Bogota,Colombia,Bogota,3688689
1,001,19 new Zika Cases in Sengkang,Sengkang,Indonesia,Sengkang,1626921
2,002,Alameda Residents Recieve Rabies vaccine,Alameda,United States,Alameda,5322737
3,003,Albany Residents Recieve Respiratory Syncytial Virus vaccine,Albany,Australia,Albany,2077963
4,003,Albany Residents Recieve Respiratory Syncytial Virus vaccine,Albany,United States,Albany,4179320
5,003,Albany Residents Recieve Respiratory Syncytial Virus vaccine,Albany,United States,Albany,5106834
6,003,Albany Residents Recieve Respiratory Syncytial Virus vaccine,Albany,United States,Albany,5322850
7,003,Albany Residents Recieve Respiratory Syncytial Virus vaccine,Albany,United States,Albany,5710756
8,004,Antipolo under threat from Zika Virus,Antipolo,Philippines,Antipolo,1730501
9,005,Arhus is infested with Bronchitis,Arhus,Denmark,Arhus,2624652


In [58]:
merged[merged.duplicated(subset=["headlineid"], keep=False)]

,headlineid,headline,city,countryname,cityname,cityid
3,003,Albany Residents Recieve Respiratory Syncytial Virus vaccine,Albany,Australia,Albany,2077963
4,003,Albany Residents Recieve Respiratory Syncytial Virus vaccine,Albany,United States,Albany,4179320
5,003,Albany Residents Recieve Respiratory Syncytial Virus vaccine,Albany,United States,Albany,5106834
6,003,Albany Residents Recieve Respiratory Syncytial Virus vaccine,Albany,United States,Albany,5322850
7,003,Albany Residents Recieve Respiratory Syncytial Virus vaccine,Albany,United States,Albany,5710756
14,010,Authorities are Worried about the Spread of Chickenpox in Richmond,Richmond,Australia,Richmond,2151649
15,010,Authorities are Worried about the Spread of Chickenpox in Richmond,Richmond,Canada,Richmond,6122085
16,010,Authorities are Worried about the Spread of Chickenpox in Richmond,Richmond,United Kingdom,Richmond,2639389
17,010,Authorities are Worried about the Spread of Chickenpox in Richmond,Richmond,New Zealand,Richmond,2183472
18,010,Authorities are Worried about the Spread of Chickenpox in Richmond,Richmond,United States,Richmond,4263681


> 📝 **Note**: We found 176 headlines that were matched with multiple cities the have the same name. For example, There were 3 different **"Rome"** city, one in the Italy and another two in the US. Thus, the headline **"Authorities are Worried about the Spread of Mad Cow Disease in Rome"** got three duplicates.
> What we need to do is to 

### US counties extraction

In [ ]:
df["us_county"] = df.headline.apply(func=lambda x: get_matched_uscounty(x))

print(df.shape)
df.head(10)

### US States extraction

In [ ]:
df["us_state"] = df.headline.apply(func=lambda x: get_matched_usstate(x))
print(df.shape)
df.head(10)


### Countries extraction

In [ ]:
df["country"] = df["headline"].apply(func=lambda x: get_matched_country(x))
print(df.shape)
df.head(10)

In [ ]:
df[df.country.notnull()]

In [51]:
df.merge(
    right=city[["countryname", "cityname", "cityid"]],
    how="left",
    left_on="city",
    right_on="cityname",
    suffixes=("_headline", "_city"),
)

,headlineid,headline,city,countryname,cityname,cityid
0,000,18 new Zika Cases in Bogota,Bogota,Colombia,Bogota,3688689
1,001,19 new Zika Cases in Sengkang,Sengkang,Indonesia,Sengkang,1626921
2,002,Alameda Residents Recieve Rabies vaccine,Alameda,United States,Alameda,5322737
3,003,Albany Residents Recieve Respiratory Syncytial Virus vaccine,Albany,Australia,Albany,2077963
4,003,Albany Residents Recieve Respiratory Syncytial Virus vaccine,Albany,United States,Albany,4179320
5,003,Albany Residents Recieve Respiratory Syncytial Virus vaccine,Albany,United States,Albany,5106834
6,003,Albany Residents Recieve Respiratory Syncytial Virus vaccine,Albany,United States,Albany,5322850
7,003,Albany Residents Recieve Respiratory Syncytial Virus vaccine,Albany,United States,Albany,5710756
8,004,Antipolo under threat from Zika Virus,Antipolo,Philippines,Antipolo,1730501
9,005,Arhus is infested with Bronchitis,Arhus,Denmark,Arhus,2624652


In [ ]:
df

In [ ]:
for ct_idx, ct in enumerate(countries.countryname):
    for st_idx, st in enumerate(df.headline):
        if ct in st:
            print(f"Headline: {st}\t\t\tCountry: {ct}")
            break

In [ ]:
suffix = [
    "County",
    "Municipality",
    "Municipio",
    "Census Area",
    "City and Borough",
    "Borough",
    "Parish",
    "City",
]

for s in suffix:
    print(f"\nSuffix: {s}")
    for headline in df.headline:
        print(f"\t{headline}") if s in headline else None

In [ ]:
city_regexs = []

for city in cities.name:
    #     r1 = '\\b'+city+'\\b'
    #     city_regexs.append(r1)
    r2 = city
    city_regexs.append(r2)
for state in states.name:
    r3 = "\\b" + state + "\\b"
    city_regexs.append(r3)
for county in counties.county:
    r4 = "\\b" + county + "\\b"
    city_regexs.append(r4)

ind = []
city = []
hline = []

for val in [
    "Mala",
    "Bron",
    "Viru",
    "Pati",
    "Rota",
    "Will",
    "Green",
    "\\b" + "Will" + "\\b",
]:
    for regex in city_regexs:
        if regex == val:
            city_regex = city_regexs.remove(val)

for regex in city_regexs:
    compiled_city = re.compile(regex)

    for index, headline in enumerate(df.headline):
        match = compiled_city.search(headline)
        if match is not None:
            start, end = match.start(), match.end()
            matched_string = headline[start:end]
            if len(matched_string) > 3:
                ind.append(index)
                city.append(matched_string)
                hline.append(headline)
#                 print(index,matched_string, '<<<', headline)

# Create dataframe of matched results and sort values by headline_no
matched = {"headline_no": ind, "headline": hline, "city": city}
matched_cities = pd.DataFrame(matched)
matched_cities = matched_cities.sort_values(by="headline_no").reset_index(drop=True)

matched_cities.head(n=15)

In [ ]:
cities[cities.name == "York"]

**Note: We found some interesting patterns of matched results.**

**For example**
* There were **three** matched results on a headline "Could Zika Reach New York City?"
* The first result was "New York City" which was a correct matched. 
* The rest two were "York" which could either be matched but why did we have two "York"?
* Check out on the `cities` dataframe then we found that there are two different locations of "York", one in the US and another one in the GB.

In [ ]:
# How many headline had matched result more than 1?
print("*********" * 10)
print("**** Checked duplicates ****")
print("*********" * 10)
print("\n")
print(
    str(
        len(
            matched_cities.headline.value_counts()[
                matched_cities.headline.value_counts() > 1
            ]
        )
    )
    + "/650 headlines had matched result more than 1."
)

# Drop duplicates of matched city names
matched_cities_uniq = matched_cities.drop_duplicates()

print("\n")
print(
    "Note: There were still number of headlines which had matched result more than 1. However, among those unique matched city names, only the longest string of city is the correct matched"
)
print("\n")

print("*********" * 10)
print("**** Dropped duplicates ****")
print("*********" * 10)
print("\n")
print(
    str(
        len(
            matched_cities_uniq.headline.value_counts()[
                matched_cities_uniq.headline.value_counts() > 1
            ]
        )
    )
    + "/650 headlines had matched result more than 1 after dropped duplicates."
)
print("\n")
print(
    matched_cities_uniq.headline.value_counts()[
        matched_cities_uniq.headline.value_counts() > 1
    ].head(10)
)
print("\n")


# Make a list of headlines that had matched result more than 1.
redun_headlines = matched_cities_uniq.headline.value_counts()[
    matched_cities_uniq.headline.value_counts() > 1
].index.tolist()
# Keep only the longest matched cities
for hl in redun_headlines:
    cities_to_compare = matched_cities_uniq.city[matched_cities_uniq.headline == hl]
    length_str = [
        (len(city), index)
        for index, city in list(zip(cities_to_compare.index, cities_to_compare))
    ]
    for length, index in length_str:
        if (length, index) != max(length_str):
            matched_cities_uniq.drop(index, axis=0, inplace=True)

print("\n")
print("*********" * 10)
print("**** Filtered only the longest matched cities ****")
print("*********" * 10)
print("\n")
print(
    str(
        len(
            matched_cities_uniq.headline.value_counts()[
                matched_cities_uniq.headline.value_counts() > 1
            ]
        )
    )
    + "/650 headlines had matched result more than 1 after filtered the longest."
)
matched_cities_uniq = matched_cities_uniq.reset_index(drop=False).set_index(
    "headline_no", drop=True
)
display(matched_cities_uniq.head())

In [ ]:
#  Add city column to df dataframe
df = df.join(matched_cities_uniq[["city"]])

# Manually fill the nulls in city column.
for i in df[df.city.isnull()].index:
    df.loc[i, "city"] = "Cebu "  # <<<  Zika infects pregnant woman in Cebu
    df.loc[i, "city"] = "Antigua "  # <<<  Spanish Flu Sighted in Antigua
    df.loc[i, "city"] = (
        "Rio De Janeiro"  # <<<  Carnival under threat in Rio De Janeiro due to Zika outbreak
    )
    df.loc[i, "city"] = "Oton"  # <<<  Zika case reported in Oton
    df.loc[i, "city"] = "Maka"  # <<<  Maka City Experiences Influenza Outbreak
    df.loc[i, "city"] = "Mcallen"  # <<<  More Zika patients reported in Mcallen
    df.loc[i, "city"] = (
        "Mclean"  # <<<  More people in Mclean are infected with Hepatitis A every year
    )
    df.loc[i, "city"] = "Sussex"  # <<<  Malaria Exposure in Sussex
    df.loc[i, "city"] = "Greenwich"  # <<<  Greenwich Establishes Zika Task Force
    df.loc[i, "city"] = "Yulee"  # <<<  Yulee takes a hit from Spreading Sickness
    df.loc[i, "city"] = (
        "Boucau"  # <<<  More people in Boucau are infected with HIV every year
    )
    df.loc[i, "city"] = "Manhasset"  # <<<  Bronchitis Outbreak in Manhasset
    df.loc[i, "city"] = "Padre Las Casas"  # <<<  Zika Troubles come to Padre Las Casas
    df.loc[i, "city"] = "Destin"  # <<<  Outbreak of Zika in Destin
    df.loc[i, "city"] = (
        "Gympie"  # <<<  Gympie Patient in Critical Condition after Contracting Chlamydia
    )
    df.loc[i, "city"] = "Druid Hills"  # <<<  Spike of Meningitis Cases in Druid Hills
    df.loc[i, "city"] = (
        "Magnolia"  # <<<  More Patients in Magnolia are Getting Diagnosed with Malaria
    )
    df.loc[i, "city"] = (
        "Penal"  # <<<  Rumors about Syphilis spreading in Penal have been refuted
    )
    df.loc[i, "city"] = "Lisbon"  # <<<  Spanish Flu Outbreak in Lisbon
    df.loc[i, "city"] = "Madrid"  # <<<  Spanish Flu Spreading through Madrid
    df.loc[i, "city"] = "Belvoir"  # <<<  Fort Belvoir tests new cure for Hepatitis C
    df.loc[i, "city"] = (
        "Oak Brook"  # <<<  More people in Oak Brook are infected with Respiratory Syncytial Virus every year
    )
    df.loc[i, "city"] = "Hutchins"  # <<<  Outbreak of Zika in Hutchins
    df.loc[i, "city"] = "Longwood"  # <<<  Longwood volunteers spreading Zika awareness
    df.loc[i, "city"] = "Quixere"  # <<<  Zika symptoms spotted in Quixere
    df.loc[i, "city"] = "Davos"  # <<<  Measles Hits Davos
    df.loc[i, "city"] = (
        "Morehead City"  # <<<  Spike of Hepatitis E Cases in Morehead City
    )
    df.loc[i, "city"] = "Alvorad"  # <<<  Outbreak of Zika in Alvorada
    df.loc[i, "city"] = "Dangriga"  # <<<  Zika arrives in Dangriga
    df.loc[i, "city"] = (
        "Maynard"  # <<<  More Patients in Maynard are Getting Diagnosed with Syphilis
    )
    df.loc[i, "city"] = "Antioquia"  # <<<  Zika case reported in Antioquia
    df.loc[i, "city"] = "Pismo Beach"  # <<<  Chikungunya has not Left Pismo Beach
    df.loc[i, "city"] = "La Joya"  # <<<  Zika spreads to La Joya
#     print("df.loc[i,'city'] = '' #<<< ",df.loc[i,'headline'] ,"\n")
df.info()

In [ ]:
df.head(50)

In [ ]:
# Save to csv file
# df.to_csv("cities_in_headline.csv", index=None)